# Module 2 Homework

## Import packages and set constants

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import requests

### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipos_filings = pd.read_html(response.text)

ipos_filings = ipos_filings[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     330 non-null    object
 1   Symbol          330 non-null    object
 2   Company Name    330 non-null    object
 3   Price Range     330 non-null    object
 4   Shares Offered  330 non-null    object
dtypes: object(5)
memory usage: 13.0+ KB


/tmp/ipykernel_535/2727273146.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipos_filings = pd.read_html(response.text)


In [3]:
# Convert to datetime
ipos_filings["Filing Date"] = pd.to_datetime(ipos_filings["Filing Date"], format="%b %d, %Y")

In [4]:
data_types = {
    "Filing Date": "datetime64[ns]",
    "Symbol": "string",
    "Company Name": "string",
    "Price Range": "string",
    "Shares Offered": "float64"
}

ipos_filings["Shares Offered"] = ipos_filings["Shares Offered"].replace("-", np.NaN)

ipos_filings = ipos_filings.astype(data_types)

In [5]:
ipos_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-14,ICON,Icon Energy Corp.,$4.00 - $6.00,1250000.0
1,2024-05-13,FCHS,"First Choice Healthcare Solutions, Inc.",$5.00,1200000.0
2,2024-05-13,KTA,Kingtitan Technology Limited,$4.00 - $6.00,3750000.0
3,2024-05-13,GALE,Galleinphi Inc.,$4.00 - $6.00,3000000.0
4,2024-05-13,CHEB,Chenghe Acquisition II Co.,$10.00,7500000.0


In [6]:
import re

def get_avg_price(input_string):
    matches = re.findall(r'(\d+.\d+)', input_string)
    avg_price = np.NaN
    if len(matches) > 1:
        avg_price = (float(matches[0]) + float(matches[1])) / 2
    elif len(matches) == 1:
        avg_price = float(matches[0])

    return avg_price


In [7]:
ipos_filings["Avg_price"] = ipos_filings["Price Range"].apply(lambda x: get_avg_price(x))

In [8]:
ipos_filings["Shares_offered_value"] = ipos_filings["Shares Offered"] * ipos_filings["Avg_price"]

In [9]:
ipos_filings["Day_of_week"] = ipos_filings["Filing Date"].dt.dayofweek
ipos_filings["Year"] = ipos_filings["Filing Date"].dt.year

In [10]:
ipos_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value,Day_of_week,Year
0,2024-05-14,ICON,Icon Energy Corp.,$4.00 - $6.00,1250000.0,5.00,6250000.0,1,2024
1,2024-05-13,FCHS,"First Choice Healthcare Solutions, Inc.",$5.00,1200000.0,5.00,6000000.0,0,2024
2,2024-05-13,KTA,Kingtitan Technology Limited,$4.00 - $6.00,3750000.0,5.00,18750000.0,0,2024
3,2024-05-13,GALE,Galleinphi Inc.,$4.00 - $6.00,3000000.0,5.00,15000000.0,0,2024
4,2024-05-13,CHEB,Chenghe Acquisition II Co.,$10.00,7500000.0,10.00,75000000.0,0,2024
...,...,...,...,...,...,...,...,...,...
325,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00,13500000.0,1,2020
326,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00,50000000.0,1,2020
327,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00,22500000.0,0,2019
328,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00,17200000.0,4,2019


In [90]:
ipos_filings[(ipos_filings["Year"]==2023) & (ipos_filings["Day_of_week"]==4)]["Shares_offered_value"].sum()

281325000.0

### Question 2:  IPOs "Fixed days hold" strategy

**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**

In [17]:
def read_ipos_url(url: str, headers: dict) -> pd.DataFrame:

    response = requests.get(url, headers=headers)
    ipo_dfs = pd.read_html(response.text)

    return ipo_dfs[0]


In [22]:
# Read IPOs
# 2023
ipos_2023 = read_ipos_url("https://stockanalysis.com/ipos/2023/", headers)
# 2024
ipos_2024 = read_ipos_url("https://stockanalysis.com/ipos/2024/", headers)

# Combine dataframes
ipos = pd.concat([ipos_2023, ipos_2024])
# Convert to datetime
ipos["IPO Date"] = pd.to_datetime(ipos["IPO Date"], format="%b %d, %Y")
# Show first rows
ipos.head()

/tmp/ipykernel_535/3982203942.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)
/tmp/ipykernel_535/3982203942.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$3.10,-22.50%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.08,-22.06%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.56,-86.11%


In [20]:
ipos

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$3.10,-22.50%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.08,-22.06%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.56,-86.11%
...,...,...,...,...,...,...
67,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.80,2.13%
68,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$2.43,-59.50%
69,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$2.74,-31.50%
70,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$28.60,36.19%


In [37]:
# Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024
tickers = ipos[ipos["IPO Date"] < "2024-03-01"]["Symbol"].unique()


In [57]:
stocks_df = pd.DataFrame({'A' : []})
for idx, ticker in enumerate(tickers):
    print(idx, ticker)
    if ticker == "RYZB":
        continue
    elif ticker == "IBAC":
        ticker = "IBACU"
    elif ticker == "PTHR":
        ticker = "HOVR"

    # Load historical daily prices
    history_prices = yf.download(
        tickers = ticker,
        period = "max",
        interval = "1d")

    # generate features for historical prices, and what we want to predict
    history_prices["Ticker"] = ticker
    history_prices["Year"]= history_prices.index.year
    history_prices["Month"] = history_prices.index.month
    history_prices["Weekday"] = history_prices.index.weekday
    history_prices["Date"] = history_prices.index.date

    # historical returns
    for i in range(1, 31):
        history_prices["growth_future_" + str(i) + "d"] = history_prices["Adj Close"].shift(-i) / history_prices["Adj Close"]


    if stocks_df.empty:
        stocks_df = history_prices
    else:
        stocks_df = pd.concat([stocks_df, history_prices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed

0 IROH
1 LGCB



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2 ZKH
3 BAYA
4 INHD


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

177 JL
178 KSPI
179 JVSA


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


180 PSBD
181 CCTG
182 SYNX


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

183 SDHC
184 ROMA


In [47]:
stocks_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticker', 'Year',
       'Month', 'Weekday', 'Date', 'growth_future_1d', 'growth_future_2d',
       'growth_future_3d', 'growth_future_4d', 'growth_future_5d',
       'growth_future_6d', 'growth_future_7d', 'growth_future_8d',
       'growth_future_9d', 'growth_future_10d', 'growth_future_11d',
       'growth_future_12d', 'growth_future_13d', 'growth_future_14d',
       'growth_future_15d', 'growth_future_16d', 'growth_future_17d',
       'growth_future_18d', 'growth_future_19d', 'growth_future_20d',
       'growth_future_21d', 'growth_future_22d', 'growth_future_23d',
       'growth_future_24d', 'growth_future_25d', 'growth_future_26d',
       'growth_future_27d', 'growth_future_28d', 'growth_future_29d',
       'growth_future_30d'],
      dtype='object')

In [48]:
stocks_df

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
0,10.050,10.050,10.010,10.0100,10.0100,16700,IROH,2024,2,4,...,1.000999,1.000999,1.000999,1.000999,1.000999,1.000999,1.000999,1.000999,1.000999,1.000999
1,10.020,10.020,10.020,10.0200,10.0200,5200,IROH,2024,2,1,...,0.999002,0.999002,0.999002,0.999002,0.999002,0.999002,0.999002,0.999002,0.999002,0.999002
2,10.020,10.020,10.015,10.0150,10.0150,98600,IROH,2024,2,2,...,0.996505,0.996505,0.996505,0.996505,0.996505,0.996505,0.996505,0.996505,0.996505,0.996505
3,10.020,10.020,10.020,10.0200,10.0200,5600,IROH,2024,2,3,...,0.996008,0.996008,0.996008,0.996008,0.996008,0.996008,0.996008,0.996008,0.996008,0.996008
4,10.020,10.020,10.010,10.0100,10.0100,14800,IROH,2024,2,4,...,0.997003,0.997003,0.997003,0.997003,0.997003,0.997003,0.997003,0.997003,0.997003,0.997003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39196,0.739,0.739,0.710,0.7110,0.7110,32900,ROMA,2024,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39197,0.740,0.810,0.715,0.7430,0.7430,273600,ROMA,2024,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39198,0.739,0.780,0.660,0.7080,0.7080,187800,ROMA,2024,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39199,0.720,0.730,0.680,0.7050,0.7050,82100,ROMA,2024,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
growth_columns = ["growth_future_" + str(i) + "d" for i in range(1, 31)]
stocks_df[growth_columns].describe()

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,39022.000000,38838.000000,38654.000000,38470.000000,38286.000000,38102.000000,37918.000000,37734.000000,37550.000000,37366.000000,...,35342.000000,35158.000000,34974.000000,34790.000000,34606.000000,34423.000000,34240.000000,34057.000000,33874.000000,33691.000000
mean,0.999790,0.999773,0.999794,0.999835,0.999831,0.999904,0.999831,0.999781,0.999775,0.999777,...,0.994392,0.993742,0.992975,0.992318,0.991736,0.991434,0.991167,0.990789,0.990481,0.990321
std,0.097619,0.137665,0.168910,0.194816,0.217575,0.240508,0.257681,0.274724,0.292829,0.310286,...,0.428038,0.438101,0.447628,0.456381,0.466407,0.479512,0.492805,0.509346,0.528560,0.550482
min,0.028674,0.003584,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,...,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729
25%,0.977718,0.966825,0.957895,0.949926,0.942509,0.935010,0.928571,0.921826,0.915694,0.909091,...,0.843125,0.837472,0.832434,0.825128,0.821328,0.815139,0.811795,0.807500,0.803562,0.799304
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.997168,0.996593,0.996209,0.995922,0.995575,0.994633,0.994163,0.994077,0.993040,0.992943
75%,1.014859,1.020172,1.025170,1.028302,1.031772,1.034121,1.036036,1.039225,1.040816,1.043011,...,1.057246,1.057826,1.057700,1.058397,1.058023,1.057560,1.059893,1.061728,1.063272,1.063811
max,6.826923,6.971311,7.625593,8.650538,8.952632,10.309092,10.183544,10.380646,9.464706,9.105319,...,12.634409,11.491442,12.486842,12.284789,12.661000,15.025219,14.782093,19.655310,22.918785,27.198465


In [72]:
d = stocks_df[growth_columns].describe().reset_index()

In [78]:
d[d["index"]=="75%"].melt(id_vars="index").sort_values(by="value", ascending=False)

,index,variable,value
29,75%,growth_future_30d,1.063811
28,75%,growth_future_29d,1.063272
27,75%,growth_future_28d,1.061728
26,75%,growth_future_27d,1.059893
23,75%,growth_future_24d,1.058397
24,75%,growth_future_25d,1.058023
21,75%,growth_future_22d,1.057826
22,75%,growth_future_23d,1.057700
25,75%,growth_future_26d,1.057560
20,75%,growth_future_21d,1.057246


In [68]:
stocks_df.agg(["min", "max", np.quant)])

TypeError: quantile() missing 2 required positional arguments: 'a' and 'q'